In [1]:
import sys


class image_interpreter():

    def __init__(self,red,green,blue):
        self.red = red
        self.blue = blue
        self.green = green
              
    def draw_on_image(self,image,cordinates_list=[[]]):
        start_t = time.time()
        image = cv2.line(image,(cordinates_list[0][0],cordinates_list[0][1]),(cordinates_list[1][0],cordinates_list[1][1]),(self.blue,self.green,self.red),1)    
        end_t = time.time()
#         print("draw_on_image: "+str(end_t-start_t))
        return image
    
    def display_image(self,image_name,window_name):
        start_t = time.time()
        cv2.imshow("view",image_name)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        end_t = time.time()
#         print("display_image: "+str(end_t-start_t))
    def draw_primary_row_chunks(self,image,height,width,horizontal_chunk_index):
        start_t = time.time()
        img = cv2.imread(image)
        resized_image = cv2.resize(img, (width,height)) 
        for index in horizontal_chunk_index:
            image=self.draw_on_image(resized_image,[[0,index],[width,index]])
        self.display_image(image,"view")
        end_t = time.time()
#         print("draw_primary_row_chunks: "+str(end_t-start_t))   
    def draw_table(self,image,data_dict):
        start_t = time.time()
        img = cv2.imread(image)
        resized_image = cv2.resize(img, (data_dict["width"],data_dict["height"]))
        for inner_dict in data_dict["chunk"]:
            start = inner_dict['start']
            end = inner_dict['end']
            column_list = inner_dict['columns']
            row_list = inner_dict['rows']                                                   
            image=self.draw_on_image(resized_image,[[0,start],[data_dict["width"],start]])
                                                               
            for column in column_list:
                image=self.draw_on_image(image,[[column,start],[column,end]])
             
            for row in row_list:
                 image=self.draw_on_image(image,[[0,row],[data_dict["width"],row]])                                              
        
        self.display_image(image,"view")    
        end_t = time.time() 
#         print("draw_table: "+str(end_t-start_t))
    def print_output_data(self,data):
        start_t = time.time()
        for primary_row in data:
            for secondary_row in primary_row:
                for secondary_column in secondary_row:
                    if type(secondary_column) is dict:
                        if secondary_column['text'] is '':
                            sys.stdout.write('\t')
                        else:
                            sys.stdout.write(secondary_column['text'].replace("\n"," ")+"\t")
                sys.stdout.write("\n")
            sys.stdout.write("\n\n")
        end_t = time.time()
#         print("print_output_data: "+str(end_t-start_t))
    
    

In [2]:
#level3

#imports
import cv2
import numpy as np
np.set_printoptions(threshold=np.inf)
import time

class data_chunking_level_3():


    def __init__(self,height,width):
        self.height=height
        self.width=width
    
    def calculate_vector(self,gray,inner,start,end):
        start_t = time.time()
        vector = []
        if inner is False:
            vector = gray.mean(axis=1)
        else:
            primary_gray = np.array(gray.tolist())
            primary_gray = primary_gray[start:end,:]
            primary_gray[primary_gray>=120] = 255
            vector= primary_gray.mean(axis=0)
        end_t = time.time()
#         print("calculate_vector: "+str(end_t-start_t)) 
        return vector

    def primary_segmentation(self,gray,row_vector,primary):
        start_t = time.time()
        
        self.primary_horizontal_skip_pixel=primary[0]
        self.primary_horizontal_threshold=primary[1]
        self.primary_horizontal_line_threshold=primary[2]
        horizontal_chunks_index = []
        
        line_detect_temp =  np.array(gray.tolist())
        line_detect_temp[line_detect_temp<=self.primary_horizontal_line_threshold] = 1
        line_detect_temp[line_detect_temp>self.primary_horizontal_line_threshold] = 0
        row_mean = line_detect_temp[:,int(self.width/3):int(2*self.width/3)].mean(axis= 1)
        horizontal_chunks_index.extend(np.where(row_mean==1)[0])
        
        white_space_temp= np.array(row_vector.tolist())
        line_index=np.where(white_space_temp>=self.primary_horizontal_threshold)[0]
        for index in range(len(line_index)):
            temp_value=line_index[index:index+self.primary_horizontal_skip_pixel]-line_index[index]
            if temp_value[len(temp_value)-1]==self.primary_horizontal_skip_pixel:
                 horizontal_chunks_index.append(line_index[index])
        horizontal_chunks_index=np.sort(np.array(horizontal_chunks_index)).tolist()
        
        end_t = time.time() 
#         print("primary_segmentation: "+str(end_t-start_t))
        return horizontal_chunks_index
    
    def accumulate_whitespaces(self,horizontal_chunks_index):
        start_t = time.time()
        deviation_index=0
        while deviation_index <= len(horizontal_chunks_index)-2:
            if horizontal_chunks_index[deviation_index] - horizontal_chunks_index[deviation_index+1] == -1:
                del horizontal_chunks_index[deviation_index]
            else:
                deviation_index= deviation_index+1
                continue
        end_t = time.time()
#         print("accumulate_whitespaces: "+str(end_t-start_t))
        return horizontal_chunks_index


    
    def print_secondary_columns(self,column_vector,secondary_vertical,inner_dict):
        start_t = time.time()
        self.secondary_vertical_skip_pixel=secondary_vertical[0]
        self.secondary_vertical_threshold=secondary_vertical[1]
        self.secondary_vertical_line_threshold=secondary_vertical[2]
        
        line_detect_temp =  np.array(column_vector.tolist())
        line_detect_temp[line_detect_temp<=self.primary_horizontal_line_threshold] = 1
        line_detect_temp[line_detect_temp>self.primary_horizontal_line_threshold] = 0
        mean = line_detect_temp.mean()
        inner_dict["columns"].extend(np.where(mean==1)[0].tolist())
        
        column_flag=False
        whitespace_start_value=0
        for column in range(self.width):
            if column <= self.width-self.secondary_vertical_skip_pixel:
                value=np.mean(column_vector[column:column+self.secondary_vertical_skip_pixel])
                if value >=self.secondary_vertical_threshold:
                    if not column_flag:
                        column_flag=True
                        whitespace_start_value=column
                else:
                    if column_flag:
                        inner_dict["columns"].append(int((whitespace_start_value+column)/2))
                        column_flag=False
        return inner_dict
    
    def print_secondary_rows(self,start_index,end_index,row_vector,secondary_horizontal,inner_dict):
        start_t = time.time()
        self.secondary_horizontal_skip_pixel=secondary_horizontal[0]
        self.secondary_horizontal_threshold=secondary_horizontal[1]
        self.secondary_horizontal_line_threshold=secondary_horizontal[2]

        line_detect_temp =  np.array(row_vector.tolist())
        line_detect_temp[line_detect_temp<=self.secondary_horizontal_line_threshold] = 1
        line_detect_temp[line_detect_temp>self.secondary_horizontal_line_threshold] = 0
        mean = line_detect_temp.mean()
        inner_dict["rows"].extend(np.where(mean==1)[0].tolist())
                
        row_flag=False  
        for row in range(start_index,end_index):
            if row<= end_index -self.secondary_horizontal_skip_pixel:
                value=np.mean(row_vector[row:row+self.secondary_horizontal_skip_pixel])
                if value >= self.secondary_horizontal_threshold:
                    if not row_flag:
                        row_flag=True
                        whitespace_start_value=row
                else:
                    if row_flag:
                        inner_dict["rows"].append(int((whitespace_start_value+row)/2))
                    row_flag=False
        end_t = time.time() 
#         print("print_secondary_rows: "+str(end_t-start_t))
        return inner_dict
    

ImportError: No module named 'cv2'

In [ ]:
#level 2

#imports
import cv2
import numpy as np
np.set_printoptions(threshold=np.inf)

class data_chunking_level_2():
    
    def __init__(self,height,width,skip_color_start,skip_color_end):
        self.height=height
        self.width=width
        self.skip_color_start=skip_color_start
        self.skip_color_end=skip_color_end
        self.image=image
        self.d3=data_chunking_level_3(height,width)
    
    
    def read_and_resize_image(self,image):
        start_t = time.time()
        img = cv2.imread(image)
        resized_image = cv2.resize(img, (self.width,self.height))
        gray = cv2.cvtColor(resized_image,cv2.COLOR_BGR2GRAY)
        gray[np.logical_and(gray>=self.skip_color_start,gray<= self.skip_color_end)]=255
        end_t = time.time() 
#         print("read_and_resize_image: "+str(end_t-start_t))
        return gray,resized_image
    
    
    def draw_segmented_rows(self,gray,primary,resized_image):
        start_t = time.time()
        row_vector=self.d3.calculate_vector(gray,False,0,0)
        horizontal_chunks_index=self.d3.primary_segmentation(gray,row_vector,primary)
        horizontal_chunks_index = self.d3.accumulate_whitespaces(horizontal_chunks_index)
        end_t = time.time() 
#         print("draw_segmented_rows: "+str(end_t-start_t))
        return horizontal_chunks_index,row_vector,resized_image

    def draw_table_in_chunks(self,gray,secondary_vertical,secondary_horizontal,horizontal_chunks_index,row_vector,resized_image):
        start_t = time.time()
        height_index = 0
        data = []
        while height_index <= len(horizontal_chunks_index) - 2:
            inner_dict = {"start":0,"end":0,"columns":[],"rows":[]}
            start=height_index
            end=height_index+1
            start_index=horizontal_chunks_index[start]
            end_index=horizontal_chunks_index[end]
            inner_dict["start"] = start_index
            inner_dict["end"] = end_index
            column_vector=self.d3.calculate_vector(gray,True,start_index,end_index)
            inner_dict=self.d3.print_secondary_columns(column_vector,secondary_vertical,inner_dict)
            inner_dict=self.d3.print_secondary_rows(start_index,end_index,row_vector,secondary_horizontal,inner_dict) 
            height_index = end
            data.append(inner_dict)
        end_t = time.time() 
#         print("draw_table_in_chunks: "+str(end_t-start_t))
        return resized_image,data
    
    def display_image(self,image_name,window_name):
        start_t = time.time()
        cv2.imshow(window_name,image_name)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        end_t = time.time() 
#         print("display_image: "+str(end_t-start_t))

In [ ]:
#level1
class data_chunking_level_1():

    
    def __init__(self,height,width,skip_color_start,skip_color_end):
        self.d2=data_chunking_level_2(height,width,skip_color_start,skip_color_end)
    
    def draw_row_chunks_on_image_file(self,image,primary):
        start_t = time.time()
        gray,resized_image=self.d2.read_and_resize_image(image)
        horizontal_chunks_index,row_vector,resized_image = self.d2.draw_segmented_rows(gray,primary,resized_image)
        end_t = time.time() 
#         print("draw_row_chunks_on_image_file: "+str(end_t-start_t))
        return horizontal_chunks_index,gray
    
    def draw_table_on_image_file(self,image,primary,secondary_vertical,secondary_horizontal):
        start_t = time.time()
        data_dict = {"height":0,"width":0,"chunk":[]}
        gray,resized_image=self.d2.read_and_resize_image(image)
        horizontal_chunks_index,row_vector,resized_image = self.d2.draw_segmented_rows(gray,primary,resized_image)
        resized_image,data = self.d2.draw_table_in_chunks(gray,secondary_vertical,secondary_horizontal,horizontal_chunks_index,row_vector,resized_image)
        data_dict["height"] = height
        data_dict["width"] = width 
        data_dict["chunk"] = data
        end_t = time.time()
        print("draw_table_on_image_file: "+str(end_t-start_t))
        return data_dict,gray
        

In [ ]:
#optical image recognition
# import cv2
# from google.cloud import vision
# import io
# import numpy as np
# from PIL import Image as PI
# from pytesseract import image_to_string
# import pyocr
# import pyocr.builders

# class optical_image_recognition():

#     def __init__(self):
#         pass

#     def crop_image(self,gray,my_dict):
#         start_t = time.time()
#         vision_client = vision.Client()
#         out=np.array(self.get_coordinates(my_dict))
#         v = 0.0
#         for primary_row in out:
#             for secondary_column in primary_row:
#                 for secondary_row in secondary_column:
#                     if type(secondary_row) is dict:
#                         crop = gray[int(secondary_row['y0']):int(secondary_row['y1']),int(secondary_row['x0']):int(secondary_row['x1'])].copy()
#                         avg=np.mean(np.mean(crop))
#                         if avg<255.0:
#                             cv2.imwrite('temp.png',crop)
#                             image_file= io.open('temp.png','rb')
#                             content = image_file.read()
#                             image = vision_client.image(content=content)
#                             s = time.time()
#                             want = image.detect_full_text()
#                             e = time.time()
#                             v= v+(e-s)
#                             print(v)
#                             secondary_row['text']=want.text
#                             image_file.close()
#         for i in range(len(out)):
#             out[i]=np.transpose(out[i])
#         end_t = time.time()
#         print("crop_image: "+str(end_t-start_t))
#         return out  

#     def get_coordinates(self,my_dict):
#         start_t = time.time()
#         main_list = []
#         for inner_dict in my_dict["chunk"]:
#             inner_list = []
#             if len(inner_dict["columns"]) > 0:
#                 column_list = inner_dict["columns"]
#                 x0 = 0
#                 for column in range(len(column_list)+1):
#                     if column == len(column_list):
#                         x1 = my_dict['width']
#                         self.get_rows(inner_dict,inner_list,x0,x1)
#                         break

#                     else:
#                         x1 = column_list[column]
#                         if x1 != x0:
#                             self.get_rows(inner_dict,inner_list,x0,x1)
#                         x0 = x1 
#             else:
#                 x0 = 0
#                 x1 = my_dict['width']
#                 val={'x0': x0,'x1':x1,'y0':inner_dict['start'],'y1':inner_dict['end'],'text':""}
#                 if val not in inner_list:
#                     inner_list.append(val)
#             main_list.append(inner_list)
#         end_t = time.time()
#         print("get_coordinates: "+str(end_t-start_t))
#         return main_list 

#     def get_rows(self,inner_dict,inner_list,x0,x1):
#         start_t = time.time()
#         if len(inner_dict["rows"]) > 0:
#             row_list = inner_dict["rows"]
#             y0 = inner_dict['start']
#             deep_inner_list=[]
#             for row in range(len(row_list)+1):
#                 if row == len(row_list):
#                     y1 = inner_dict['end']
#                     if deep_inner_list not in inner_list:
#                         inner_list.append(deep_inner_list)
#                     val={'x0': x0,'x1':x1,'y0':y0,'y1':y1,'text':""}
#                     if val not in deep_inner_list:
#                         deep_inner_list.append(val)
#                     break

#                 else:
#                     y1 = row_list[row]
#                     if y1 != y0:
#                         if deep_inner_list not in inner_list:
#                             inner_list.append(deep_inner_list)
#                         val={'x0': x0,'x1':x1,'y0':y0,'y1':y1,'text':""}
#                         if val not in deep_inner_list:
#                             deep_inner_list.append(val)
#                     y0 = y1  
#         else :
#             deep_inner_list=[]
#             if deep_inner_list not in inner_list:
#                 inner_list.append(deep_inner_list)
#             val={'x0': x0,'x1':x1,'y0':inner_dict['start'],'y1':inner_dict['end'],'text':""}
#             if val not in deep_inner_list:
#                 deep_inner_list.append(val)
#         end_t = time.time()
#         print("get_rows: "+str(end_t-start_t))     

In [ ]:
#client

height=1200
width=1000

skip_color_start=170
skip_color_end=225

blue=0
red=0
green =0

image='/images/test1-11.png'

chunking = data_chunking_level_1(height,width,skip_color_start,skip_color_end)
data_dict,gray = chunking.draw_table_on_image_file(image,[12,254,165],[9,254,-1],[5,254,165])
interpret = image_interpreter(red,green,blue)
interpret.draw_table(image,data_dict)
print(data_dict)
# oir=optical_image_recognition()
# out=oir.crop_image(gray,data_dict)
# interpret.print_output_data(out)

In [ ]:
# import PythonMagick
# import os
# from pyPdf import PdfFileReader


# file = PdfFileReader(file(input_pdf,'rb'))
# total_pages = file.gethttp://localhost:8888/notebooks/PycharmProjects/dcm/data_chunking_client.ipynb#NumPages()

# os.system("pdftk C:\\Users\\Admin\\PycharmProjects\\"+input_pdf+"BURST OUTPUT C:\\Users\\Admin\\PycharmProjects\\input%03d.pdf")

# for page in range(total_pages):
#     index = page + 1
#     img = PythonMagick.Image()
#     img.density("500")
#     img.read(+"input00%d.pdf"%index) 
#     img.write(+"input00%d.png"%index)
    

